In [ ]:
from imports_LDVAE_local import *
from scipy.stats import spearmanr
from matplotlib.colors import Normalize
warnings.filterwarnings('ignore')

In [ ]:
pre = "MH106"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
#load proof of principle results from MH80
_,base_table_path_MH80,_,_ = h.return_local_paths(drive = drive,pre = 'MH80',add_path = False)

In [ ]:
set_types = ['train','test']
celltypes = ['B',
 'CD14 Mono',
 'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']

In [ ]:
warnings.filterwarnings("ignore")
df_spearman = pd.DataFrame(columns = ['r_cyno_human','r_cyno_shifted_human',
                                      'pval_cyno_human','pval_cyno_shifted_human',
                                      'r_cyno_human_filtered','r_cyno_shifted_human_filtered',
                                      'pval_cyno_human_filtered','pval_cyno_shifted_human_filtered',
                                      'set_type','celltype'])
for set_type in set_types:
    for celltype in celltypes:
        df_temp = pd.read_csv(os.path.join(base_table_path_MH80,'MH80_df_ct_mean_expr_' + str(set_type) + '_' + str(celltype) + '.csv'),index_col=0).T
        df_temp['rank_00hr_cyno_'+ str(set_type) ] = df_temp['mean_expression_00hr_cyno_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)

        df_temp['rank_00hr_human_'+ str(set_type) ] = df_temp['mean_expression_00hr_human_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)

        df_temp['rank_00hr_cyno_shifted_'+ str(set_type) ] =  df_temp['mean_expression_00hr_cyno_shifted_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)
        df_temp.to_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_rank.csv'))
        r_cyno_human = spearmanr(df_temp['rank_00hr_cyno_'+ str(set_type)],
                                 df_temp['rank_00hr_human_'+ str(set_type)])[0]
        
        pval_cyno_human = spearmanr(df_temp['rank_00hr_cyno_'+ str(set_type)],
                                 df_temp['rank_00hr_human_'+ str(set_type)])[1]

        r_cyno_shifted_human = spearmanr(df_temp['rank_00hr_cyno_shifted_'+ str(set_type)],
                                         df_temp['rank_00hr_human_'+ str(set_type)])[0]
        pval_cyno_shifted_human = spearmanr(df_temp['rank_00hr_cyno_shifted_'+ str(set_type)],
                                         df_temp['rank_00hr_human_'+ str(set_type)])[1]
        df_temp_filtered = df_temp[df_temp['mean_expression_00hr_human_' +str(set_type)] > 0]
        

        df_temp_filtered['rank_00hr_cyno_' +str(set_type)] = df_temp_filtered['rank_00hr_cyno_' +str(set_type)].rank(method='dense',ascending = True).astype(int)
        df_temp_filtered['rank_00hr_cyno_shifted_' +str(set_type)] = df_temp_filtered['rank_00hr_cyno_shifted_' +str(set_type)].rank(method='dense',ascending = True).astype(int)
        df_temp_filtered.to_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_filtered_rank.csv'))

        df_temp_filtered.to_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_filtered_rank.csv'))

        r_cyno_human_filtered = spearmanr(df_temp_filtered['rank_00hr_cyno_'+ str(set_type)],
                                         df_temp_filtered['rank_00hr_human_'+ str(set_type)])[0]

        pval_cyno_human_filtered = spearmanr(df_temp_filtered['rank_00hr_cyno_'+ str(set_type)],
                                 df_temp_filtered['rank_00hr_human_'+ str(set_type)])[1]

        r_cyno_shifted_human_filtered = spearmanr(df_temp_filtered['rank_00hr_cyno_shifted_'+ str(set_type)],
                                         df_temp_filtered['rank_00hr_human_'+ str(set_type)])[0]
        pval_cyno_shifted_human_filtered = spearmanr(df_temp_filtered['rank_00hr_cyno_shifted_'+ str(set_type)],
                                         df_temp_filtered['rank_00hr_human_'+ str(set_type)])[1]
        df_spearman = df_spearman.append(
            pd.DataFrame([r_cyno_human,r_cyno_shifted_human,
                          pval_cyno_human,pval_cyno_shifted_human,
                          r_cyno_human_filtered,r_cyno_shifted_human_filtered,
                            pval_cyno_human_filtered,pval_cyno_shifted_human_filtered,
                          set_type,celltype]
                         ,index = ['r_cyno_human','r_cyno_shifted_human',
                                   'pval_cyno_human','pval_cyno_shifted_human',
                                   'r_cyno_human_filtered','r_cyno_shifted_human_filtered',
                                    'pval_cyno_human_filtered','pval_cyno_shifted_human_filtered',
                                   'set_type','celltype']).T
        )  

## Plot rank scatter

In [ ]:
set_type = 'train'

fig, axes = plt.subplots(len(celltypes), 2, figsize=(6, 2.5 * len(celltypes)))  


for i, celltype in enumerate(celltypes):
    df_temp_rank = pd.read_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_rank.csv'),index_col=0)
    axes[i, 0].scatter(df_temp_rank['rank_00hr_human_train'], df_temp_rank['rank_00hr_cyno_train'],s=6,color = "#022a2a",alpha =0.9)  # Replace x1, y1 with data for the first plot
    axes[i, 0].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 0].set_xlabel('A')
    axes[i, 0].set_xlabel('human rank')
    axes[i, 0].set_ylabel('cyno rank')
    

    axes[i, 1].scatter(df_temp_rank['rank_00hr_human_train'], df_temp_rank['rank_00hr_cyno_shifted_train'],s=6,color = "#022a2a",alpha =0.9)  # Replace x2, y2 with data for the second plot
    axes[i, 1].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 1].set_xlabel('human rank')
    axes[i, 1].set_ylabel('humanized cyno rank')

plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_all_ct_' + set_type+ '.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
#filtered
set_type = 'train'

fig, axes = plt.subplots(len(celltypes), 2, figsize=(6, 2.5 * len(celltypes)))  


for i, celltype in enumerate(celltypes):
    df_temp_rank = pd.read_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_filtered_rank.csv'),index_col=0)
    axes[i, 0].scatter(df_temp_rank['rank_00hr_human_train'], df_temp_rank['rank_00hr_cyno_train'],s=6,color = "#022a2a",alpha =0.9)  # Replace x1, y1 with data for the first plot
    axes[i, 0].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 0].set_xlabel('A')
    axes[i, 0].set_xlabel('human rank')
    axes[i, 0].set_ylabel('cyno rank')
    

    axes[i, 1].scatter(df_temp_rank['rank_00hr_human_train'], df_temp_rank['rank_00hr_cyno_shifted_train'],s=6,color = "#022a2a",alpha =0.9)  # Replace x2, y2 with data for the second plot
    axes[i, 1].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 1].set_xlabel('human rank')
    axes[i, 1].set_ylabel('humanized cyno rank')

plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_all_ct_' + set_type+ '_filtered_rank.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
#Filtered
set_type = 'test'
fig, axes = plt.subplots(len(celltypes), 2, figsize=(6, 2.5 * len(celltypes)))  

for i, celltype in enumerate(celltypes):
    df_temp_rank = pd.read_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_filtered_rank.csv'),index_col=0)
    axes[i, 0].scatter(df_temp_rank['rank_00hr_human_test'], df_temp_rank['rank_00hr_cyno_test'],s=6,color = "#022a2a",alpha =0.9)  # Replace x1, y1 with data for the first plot
    axes[i, 0].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 0].set_xlabel('A')
    axes[i, 0].set_xlabel('human rank')
    axes[i, 0].set_ylabel('cyno rank')
    

    axes[i, 1].scatter(df_temp_rank['rank_00hr_human_test'], df_temp_rank['rank_00hr_cyno_shifted_test'],s=6,color = "#022a2a",alpha =0.9)  # Replace x2, y2 with data for the second plot
    axes[i, 1].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 1].set_xlabel('human rank')
    axes[i, 1].set_ylabel('humanized cyno rank')


plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_all_ct_' + set_type+ 'filtered_rank.pdf'),bbox_inches='tight')

plt.show()

In [ ]:
set_type = 'test'
fig, axes = plt.subplots(len(celltypes), 2, figsize=(6, 2.5 * len(celltypes)))  

for i, celltype in enumerate(celltypes):
    df_temp_rank = pd.read_csv(os.path.join(base_table_path,pre + '_' + set_type + '_' + str(celltype) + '_df_temp_rank.csv'),index_col=0)
    axes[i, 0].scatter(df_temp_rank['rank_00hr_human_test'], df_temp_rank['rank_00hr_cyno_test'],s=6,color = "#022a2a",alpha =0.9)  # Replace x1, y1 with data for the first plot
    axes[i, 0].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 0].set_xlabel('A')
    axes[i, 0].set_xlabel('human rank')
    axes[i, 0].set_ylabel('cyno rank')
    

    axes[i, 1].scatter(df_temp_rank['rank_00hr_human_test'], df_temp_rank['rank_00hr_cyno_shifted_test'],s=6,color = "#022a2a",alpha =0.9)  # Replace x2, y2 with data for the second plot
    axes[i, 1].set_title(celltype + ' - ' + set_type + ' set',fontsize =10)
    axes[i, 1].set_xlabel('human rank')
    axes[i, 1].set_ylabel('humanized cyno rank')


plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_all_ct_' + set_type+ '.pdf'),bbox_inches='tight')

plt.show()

In [ ]:
df_spearman['r_cyno_shifted_human_filtered'] = pd.to_numeric(df_spearman['r_cyno_shifted_human_filtered'],errors='coerce')

df_spearman['r_cyno_human_filtered'] = pd.to_numeric(df_spearman['r_cyno_human_filtered'], errors='coerce')

In [ ]:
df_spearman['set_type+celltype'] = df_spearman['set_type'].astype(str) + '_' + df_spearman['celltype'].astype(str)

df_spearman['r_cyno_shifted_human_filtered'] = pd.to_numeric(df_spearman['r_cyno_shifted_human_filtered'],errors='coerce')

df_spearman['r_cyno_human_filtered'] = pd.to_numeric(df_spearman['r_cyno_human_filtered'], errors='coerce')



df_pivot_r_cyno_human_filtered = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_human_filtered')
df_pivot_r_cyno_shifted_human_filtered = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_shifted_human_filtered')

df_pivot_r_cyno_shifted_human_filtered = df_pivot_r_cyno_shifted_human_filtered.reindex(['train', 'test'])

df_pivot_r_cyno_human_filtered = df_pivot_r_cyno_human_filtered.reindex(['train', 'test'])

df_pivot_r_cyno_human_filtered = df_pivot_r_cyno_human_filtered.reindex(['train', 'test'])

df_all_filtered = df_pivot_r_cyno_human_filtered.append(df_pivot_r_cyno_shifted_human_filtered)
df_all_filtered.index =['train_human_cyno_filtered', 'test_human_cyno_filtered', 'train_human_shifted_cyno_filtered', 'test_human_shifted_cyno_filtered']
df_all_filtered.to_csv(os.path.join(base_table_path,pre + '_df_all_filtered_spearman.csv'))

In [ ]:
plt.figure(figsize=(10, 6))
ax=sns.heatmap(df_all_filtered.T, cmap="BuPu", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('Rank correlation proof principle')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

#darl middle line
middle_line_index = 2 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4) 
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_all_filtered.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_spearman['set_type+celltype'] = df_spearman['set_type'].astype(str) + '_' + df_spearman['celltype'].astype(str)

df_spearman['r_cyno_shifted_human'] = pd.to_numeric(df_spearman['r_cyno_shifted_human'],errors='coerce')

df_spearman['r_cyno_human'] = pd.to_numeric(df_spearman['r_cyno_human'], errors='coerce')



df_pivot_r_cyno_human = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_human')
df_pivot_r_cyno_shifted_human = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_shifted_human')

df_pivot_r_cyno_shifted_human = df_pivot_r_cyno_shifted_human.reindex(['train', 'test'])

df_pivot_r_cyno_human = df_pivot_r_cyno_human.reindex(['train', 'test'])

df_pivot_r_cyno_human = df_pivot_r_cyno_human.reindex(['train', 'test'])

df_all = df_pivot_r_cyno_human.append(df_pivot_r_cyno_shifted_human)

In [ ]:
df_all.index =['train_human_cyno', 'test_human_cyno', 'train_human_shifted_cyno', 'test_human_shifted_cyno']

In [ ]:
df_all.to_csv(os.path.join(base_table_path,pre + '_df_all_spearman.csv'))

In [ ]:
plt.figure(figsize=(10, 6))
ax=sns.heatmap(df_all.T, cmap="BuPu", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('Rank correlation proof principle')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

#darl middle line
middle_line_index = 2 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4) 
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_all.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_all_filtered_only_train = df_all_filtered.T[['train_human_cyno_filtered','train_human_shifted_cyno_filtered']].T
df_all_filtered_only_train.to_csv(os.path.join(base_table_path,pre + '_df_all_filtered_only_train_spearman.csv'))

In [ ]:
plt.figure(figsize=(5, 6))
ax=sns.heatmap(df_all_filtered_only_train.T, cmap="RdYlGn", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('train set')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
#darl middle line
middle_line_index = 1 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4)
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_train_filtered.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_all_only_train = df_all.T[['train_human_cyno','train_human_shifted_cyno']].T
df_all_only_train.to_csv(os.path.join(base_table_path,pre + '_df_all_only_train_spearman.csv'))

In [ ]:
plt.figure(figsize=(5, 6))
ax=sns.heatmap(df_all_only_train.T, cmap="RdYlGn", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('train set')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
#darl middle line
middle_line_index = 1 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4) 
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_train.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_all_filtered_only_test = df_all_filtered.T[['test_human_cyno_filtered','test_human_shifted_cyno_filtered']].T
df_all_filtered_only_test.to_csv(os.path.join(base_table_path,pre + '_df_all_filtered_only_test_spearman.csv'))

In [ ]:
plt.figure(figsize=(5, 6))
ax=sns.heatmap(df_all_filtered_only_test.T, cmap="RdYlGn", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('test set')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
#darl middle line
middle_line_index = 1 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4)
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_test_filtered.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_all_only_test = df_all.T[['test_human_cyno','test_human_shifted_cyno']].T
df_all_only_test.to_csv(os.path.join(base_table_path,pre + '_df_all_only_test_spearman.csv'))

In [ ]:
plt.figure(figsize=(5, 6))
ax=sns.heatmap(df_all_only_test.T, cmap="RdYlGn", annot=True, fmt=".4f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.55, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('test set')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
#darl middle line
middle_line_index = 1 
ax.vlines(middle_line_index, *ax.get_ylim(), color='black', linewidth=4) 
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_test.pdf'),bbox_inches='tight')
plt.show()

## Save session

In [ ]:

base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)